In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [43]:
!pip install spylls
!pip install contractions
!pip install hyphenate

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import spacy

2022-09-15 07:53:18.907760: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-15 07:53:18.924780: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-09-15 07:53:18.924791: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
import pandas as pd
import numpy as np

In [4]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [5]:
import tensorflow as tf

In [6]:
from scipy import sparse

In [7]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet

from sklearn.kernel_ridge import KernelRidge

from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import RANSACRegressor

from sklearn.linear_model import MultiTaskElasticNet
from sklearn.linear_model import MultiTaskLasso

In [9]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import HistGradientBoostingRegressor

In [10]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [11]:
from tqdm import tqdm
import gc
from spylls.hunspell import Dictionary
from string import punctuation
import re
import contractions
from nltk import word_tokenize
from nltk.corpus import stopwords
from itertools import chain

from hyphenate import hyphenate_word
from nltk.tokenize import sent_tokenize

In [12]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
sample_submission = pd.read_csv("data/sample_submission.csv")

In [13]:
simple_logs = pd.DataFrame()

In [14]:
def mcrmse(y_true, y_pred):
    colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=0)
    return tf.reduce_mean(tf.sqrt(colwise_mse), axis=0)

In [15]:
train.head(5)

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [16]:
test

,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


In [17]:
FEATURE_COLUMNS = ['full_text']
TARGET_COLUMNS = ['cohesion', 'syntax','vocabulary', 'phraseology', 'grammar','conventions']

In [18]:
X, Y = train[FEATURE_COLUMNS], train[TARGET_COLUMNS]


In [19]:
kfold = KFold(n_splits=5)

#### LinReg
Mean score: 0.550

In [25]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = MultiOutputRegressor(LinearRegression(), n_jobs = -1)
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")
simple_logs[]
    

Fold #0: 0.5541318246499967
Fold #1: 0.5537700014592977
Fold #2: 0.5470261004584632
Fold #3: 0.544407325985179
Fold #4: 0.5520343641116121
********************************************************************************

	Mean loss: 0.550


In [26]:
# with rounding 
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = MultiOutputRegressor(LinearRegression(), n_jobs = -1)
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    y_pred = np.round(y_pred*2) / 2
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")
    

Fold #0: 0.5709403512789716
Fold #1: 0.5726226508335558
Fold #2: 0.5635645363980998
Fold #3: 0.5632515029920215
Fold #4: 0.5698789714147965
********************************************************************************

	Mean loss: 0.568


#### ElasticNet
Mean score: .653

In [27]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    # X_train, X_test = X_train.toarray(), X_test.toarray()

    model = MultiOutputRegressor(ElasticNet(), n_jobs = -1)
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    # test_vectorized = test_vectorized.toarray()
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

Fold #0: 0.6528782028635198
Fold #1: 0.6741904406848621
Fold #2: 0.6446424251073121
Fold #3: 0.6458552901536275
Fold #4: 0.6467219977663213
********************************************************************************

	Mean loss: 0.653


#### Kernel ridge 
Mean score: 0.683

In [28]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    # X_train, X_test = X_train.toarray(), X_test.toarray()

    model = MultiOutputRegressor(KernelRidge(), n_jobs = -1)
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    # test_vectorized = test_vectorized.toarray()
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

Fold #0: 0.6930777265189026
Fold #1: 0.6750998745559542
Fold #2: 0.6843776292717582
Fold #3: 0.6704404514406113
Fold #4: 0.6934175409997669
********************************************************************************

	Mean loss: 0.683


#### Huber
Mean score: 0.5478 <br>
tfidf (1,2) best

In [40]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = MultiOutputRegressor(HuberRegressor(max_iter=100))
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #0: 0.5515620102984613


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #1: 0.5504950203190537


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #2: 0.5446898913826562


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #3: 0.5419538049081577


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #4: 0.5502569124296768
********************************************************************************

	Mean score: 0.5478


In [41]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,1))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = MultiOutputRegressor(HuberRegressor(max_iter=100))
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #0: 0.6331420093762457


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #1: 0.630056943661134


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #2: 0.626796640564493


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #3: 0.6249290599158402


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #4: 0.6326062592100771
********************************************************************************

	Mean score: 0.6295


In [42]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,3))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = MultiOutputRegressor(HuberRegressor(max_iter=100))
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #0: 0.5538161068513142


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #1: 0.5509624993104995


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #2: 0.5509353575657568


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #3: 0.5418378324264046


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #4: 0.5560860014742328
********************************************************************************

	Mean score: 0.5507


#### MultiTaskLasso
Mean score: 0.653

In [19]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    X_train, X_test = X_train.toarray(), X_test.toarray()
    
    model = MultiTaskLasso()
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    test_vectorized = test_vectorized.toarray()

    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

Fold #0: 0.6528782028635198
Fold #1: 0.6741904406848621
Fold #2: 0.6446424251073121
Fold #3: 0.6458552901536275
Fold #4: 0.6467219977663213
********************************************************************************

	Mean loss: 0.653


#### Decision Tree
Mean score: 0.807

In [16]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = DecisionTreeRegressor()
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

2022-09-15 04:36:22.455220: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-15 04:36:22.471719: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-09-15 04:36:22.471730: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-09-15 04:36:22.472148: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

Fold #0: 0.7931361523412584
Fold #1: 0.8144465691907875
Fold #2: 0.8023388564277685
Fold #3: 0.8114694681488267
Fold #4: 0.8132698138502844
********************************************************************************

	Mean loss: 0.807


#### Random Forest
Mean score: 0.569

In [ ]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = RandomForestRegressor(n_jobs = -1)
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

#### XGB
Mean score: 0.582

In [17]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = XGBRegressor(n_jobs = -1)
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

2022-09-15 04:58:14.703313: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-15 04:58:14.721896: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-09-15 04:58:14.721910: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-09-15 04:58:14.722549: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

Fold #0: 0.5816267489732073
Fold #1: 0.5901526685793419
Fold #2: 0.5822439205413895
Fold #3: 0.5774044165601089
Fold #4: 0.5763746517735759
********************************************************************************

	Mean loss: 0.582


#### LGBM
Mean score: 0.550

In [21]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = MultiOutputRegressor(LGBMRegressor(n_jobs = -1))
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

Fold #0: 0.5517973919318183
Fold #1: 0.5551591652486675
Fold #2: 0.5483291446127709
Fold #3: 0.5482365804964576
Fold #4: 0.5465152434993047
********************************************************************************

	Mean score: 0.550


In [22]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,1))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = MultiOutputRegressor(LGBMRegressor(n_jobs = -1))
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

Fold #0: 0.5569290386975966
Fold #1: 0.5516524187961335
Fold #2: 0.5498964155109362
Fold #3: 0.5458818916232535
Fold #4: 0.5478695454500438
********************************************************************************

	Mean score: 0.550


#### KNN
Mean score: 0.6645

In [31]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = KNeighborsRegressor(n_jobs = -1)
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

Fold #0: 0.671892799169405
Fold #1: 0.67549296640102
Fold #2: 0.6435011356635894
Fold #3: 0.6580667474425836
Fold #4: 0.6736422701792272
********************************************************************************

	Mean score: 0.6645


#### SVR
Mean score: 0.5507

In [29]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = MultiOutputRegressor(SVR())
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

Fold #0: 0.5527417883782569
Fold #1: 0.5598486346576218
Fold #2: 0.5466700014211566
Fold #3: 0.5447914627070536
Fold #4: 0.549511123877991
********************************************************************************

	Mean score: 0.5507


In [30]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,1))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = MultiOutputRegressor(SVR())
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

Fold #0: 0.5534128628705631
Fold #1: 0.5568149787750852
Fold #2: 0.5462140456365606
Fold #3: 0.5468043613997916
Fold #4: 0.5493505076755353
********************************************************************************

	Mean score: 0.5505


#### HistGradient

In [37]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,1))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    X_train, X_test = X_train.toarray(), X_test.toarray()

    model = MultiOutputRegressor(HistGradientBoostingRegressor())
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_vectorized = test_vectorized.toarray()
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

Fold #0: 0.5537410728071898


KeyboardInterrupt: 

### New features

In [20]:
STOPWORDS = set(stopwords.words('english'))
DICTIONARY = Dictionary.from_files('en_US')
PUNCTUATIONS = set(list(punctuation))
PUNCTUATIONS.update("`")
PUNCTUATIONS.update("'")
CONTRACTIONS = contractions.contractions_dict
CONTRACTIONS["It'll"] = "It will"
CONTRACTIONS = {key.capitalize(): value for key, value in CONTRACTIONS.items()}


def check_word(token):
    if DICTIONARY.lookup(token):
        return 1
    else:
        return 0

def replace_contractions(text):

    
    for key, value in CONTRACTIONS.items():
        # Upper-case
        text = text.replace(key, value)
        # Lower-case 
        text = text.replace(key.lower(), value.lower())

    # Remove possesives as well 
    text = text.replace("'s", "")

    # Remove unnecessary whitespaces
    text = re.sub(' +', ' ', text)
    
    return text 


def remove_empty_tokens(list_of_tokens):
    list_of_tokens = list(filter(None, list_of_tokens))
    return list_of_tokens

def check_in_dictionary(sentences):

    # If the contraction can be removed, than it means it's correct; consequently, only possibly incorrect words remain

    tokenized_sentences = word_tokenize(sentences)
    correct_tokens = [check_word(token) if (token not in PUNCTUATIONS)  else 1 for token in tokenized_sentences]

    return correct_tokens

def get_incorrect_indices(correct_word_list):
    return [idx for idx, value in enumerate(correct_word_list) if not value]

def get_incorrect_words(words_list, indices):
    return [words_list[idx] for idx in indices if words_list[idx] ]

In [21]:
nlp = spacy.load('en_core_web_lg')

In [22]:
def extract_spacy_features(texts):
    
    tokens = []
    poss = []
    tags = []
    morphs = []
    
    for doc in nlp.pipe(texts, n_process = -1):
        
        token_ = []
        pos_ = []
        tag_ = []
        morph_ = []

        for token in doc:
            token_.append(token.text)
            pos_.append(token.pos_)
            tag_.append(token.tag_)
            morph_.append(token.morph)
            
        tokens.append(token_)
        poss.append(pos_)
        tags.append(tag_)
        morphs.append(morph_)
        
    return tokens, poss, tags, morphs

In [23]:
def extract_verbform(morph):
    vbs = []
    for i in morph:
        vb = i.get('VerbForm')
        if vb != []:
            vbs.append(vb[0])
    return vbs

In [24]:
def add_new_features(X:pd.DataFrame):
    # TODO: replace w/ assign
    X['sentences'] = X.full_text.map(sent_tokenize)
    X['words'] = X.sentences.apply(lambda sentences: list(chain(*[word_tokenize(sentence) for sentence in sentences])))
    X['words'] = X.words.apply(lambda tokens: [token for token in tokens if (token not in PUNCTUATIONS and token != '')])
    X['syllables'] = X.words.apply(lambda tokens: list(chain(*[hyphenate_word(token) for token in tokens])))

    X['text_'] = X.full_text.map(replace_contractions)
    X['tokenized_sentences'] = X.text_.map(word_tokenize)
    X['correct_list'] = X.text_.map(check_in_dictionary)
    X['incorrect_indices'] = X.correct_list.map(get_incorrect_indices)
    X['incorrect_words'] = X.apply(lambda x: get_incorrect_words(x.tokenized_sentences, x.incorrect_indices), axis = 1)

    X['incorrect_words_count'] = X.incorrect_words.map(len)

    X['sent_count'] = X.sentences.map(len)
    X['word_count'] = X.words.map(len)
    X['syll_count'] = X.syllables.map(len)

    X['words_per_sentences'] = X['sent_count'] / X['word_count']
    X['syll_per_sentences'] = X['sent_count'] / X['syll_count'] 
    X['syll_per_words'] = X['word_count'] / X['syll_count']
    X['contains_question'] = X.full_text.str.contains('\?').astype(int)
    
    tokens_train, poss_, tags_, morphs_ = extract_spacy_features(X.full_text.values)

    X['poss_'] = poss_
    X['tag_'] = tags_
    X['morph_'] = morphs_

    X.poss_ = X.poss_.str.join(' ')

    X['VF'] = X.morph_.map(extract_verbform).str.join(' ')

    return X

In [25]:
add_new_features(X)
add_new_features(test)

/tmp/ipykernel_270224/1747429121.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['sentences'] = X.full_text.map(sent_tokenize)
/tmp/ipykernel_270224/1747429121.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['words'] = X.sentences.apply(lambda sentences: list(chain(*[word_tokenize(sentence) for sentence in sentences])))
/tmp/ipykernel_270224/1747429121.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

,text_id,full_text,sentences,words,syllables,text_,tokenized_sentences,correct_list,incorrect_indices,incorrect_words,...,word_count,syll_count,words_per_sentences,syll_per_sentences,syll_per_words,contains_question,poss_,tag_,morph_,VF
0,0000C359D63E,when a person has no experience on a job their...,[when a person has no experience on a job thei...,"[when, a, person, has, no, experience, on, a, ...","[when, a, per, son, has, no, ex, pe, ri, ence,...",when a person has no experience on a job their...,"[when, a, person, has, no, experience, on, a, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[23, 43, 45, 82, 103, 123, 125, 182, 213, 215,...","[explane, bassicly, learing, dont, dont, dont,...",...,839,1060,0.030989,0.024528,0.791509,0,SCONJ DET NOUN VERB DET NOUN ADP DET NOUN PRON...,"[WRB, DT, NN, VBZ, DT, NN, IN, DT, NN, PRP$, V...","[(), (Definite=Ind, PronType=Art), (Number=Sin...",Fin Fin Part Inf Inf Inf Inf Fin Inf Part Fin ...
1,000BAD50D026,Do you think students would benefit from being...,[Do you think students would benefit from bein...,"[Do, you, think, students, would, benefit, fro...","[Do, you, think, stu, dents, would, ben, e, fi...",Do you think students would benefit from being...,"[Do, you, think, students, would, benefit, fro...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[43, 73, 189, 205, 363, 397]","[ththem, attand, ththem, ththem, attand, ththe...",...,399,523,0.042607,0.032505,0.762906,1,AUX PRON VERB NOUN AUX VERB ADP AUX ADJ VERB N...,"[VBP, PRP, VB, NNS, MD, VB, IN, VBG, JJ, VB, N...","[(Mood=Ind, Tense=Pres, VerbForm=Fin), (Case=N...",Fin Inf Fin Inf Ger Inf Fin Fin Inf Fin Fin Fi...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde...","[Thomas Jefferson once states that ""it is wond...","[Thomas, Jefferson, once, states, that, ``, it...","[Thomas, Jef, fer, son, once, states, that, ``...","Thomas Jefferson once states that ""it is wonde...","[Thomas, Jefferson, once, states, that, ``, it...","[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 26, 203, 218]","[``, statthement, '', '']",...,454,590,0.037445,0.028814,0.769492,1,PROPN PROPN ADV VERB SCONJ PUNCT PRON AUX ADJ ...,"[NNP, NNP, RB, VBZ, IN, ``, PRP, VBZ, JJ, WRB,...","[(Number=Sing), (Number=Sing), (NumType=Mult),...",Fin Fin Fin Inf Part Fin Part Fin Fin Fin Inf ...


In [26]:
train.columns

Index(['text_id', 'full_text', 'cohesion', 'syntax', 'vocabulary',
       'phraseology', 'grammar', 'conventions'],
      dtype='object')

In [27]:
other_features = ['incorrect_words_count', 'sent_count', 'word_count', 'syll_count', 'words_per_sentences', 'syll_per_sentences', 'syll_per_words', 'contains_question']


#### Huber: tfidf + other features

In [90]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]

    text_vectorizer = TfidfVectorizer(ngram_range = (1,2))
    vectorized_text_train = text_vectorizer.fit_transform(X_train.full_text.tolist())
    vectorized_text_test = text_vectorizer.transform(X_test.full_text.tolist())
    vectorized_text_actual_test = text_vectorizer.transform(test.full_text.tolist())

    pos_vectorizer = TfidfVectorizer(ngram_range=(1,4))
    morph_vectorizer = TfidfVectorizer()

    vectorized_pos_train =  pos_vectorizer.fit_transform(X_train.poss_)
    vectorized_pos_test =  pos_vectorizer.transform(X_test.poss_)
    vectorized_pos_actual_test = pos_vectorizer.transform(test.poss_)

    vectorized_morph_train =  morph_vectorizer.fit_transform(X_train.VF)
    vectorized_morph_test =  morph_vectorizer.transform(X_test.VF)
    vectorized_morph_actual_test =  morph_vectorizer.transform(test.VF)

    other_features_subset_train = X_train[other_features]
    other_features_subset_test = X_test[other_features]
    other_features_subset_actual_test = test[other_features]

    X_train = sparse.hstack([
        vectorized_text_train,
        vectorized_pos_train,
        vectorized_morph_train,
        other_features_subset_train
    ])

    X_test = sparse.hstack([
        vectorized_text_test,
        vectorized_pos_test,
        vectorized_morph_test,
        other_features_subset_test
    ])


    X_actual_test = sparse.hstack([
        vectorized_text_actual_test,
        vectorized_pos_actual_test,
        vectorized_morph_actual_test,
        other_features_subset_actual_test
    ])


    model = MultiOutputRegressor(HuberRegressor())
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    
    test_prediction = model.predict(X_actual_test)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #0: 0.603293874036087


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #1: 0.5835297339852062


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #2: 0.5845026136418409


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #3: 0.5654080267131697


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #4: 0.5601884935231137
********************************************************************************

	Mean score: 0.5794


#### Huber: other features

In [88]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    pos_vectorizer = TfidfVectorizer(ngram_range=(1,4))
    morph_vectorizer = TfidfVectorizer()

    vectorized_pos_train =  pos_vectorizer.fit_transform(X_train.poss_)
    vectorized_pos_test =  pos_vectorizer.transform(X_test.poss_)
    vectorized_pos_actual_test = pos_vectorizer.transform(test.poss_)

    vectorized_morph_train =  morph_vectorizer.fit_transform(X_train.VF)
    vectorized_morph_test =  morph_vectorizer.transform(X_test.VF)
    vectorized_morph_actual_test =  morph_vectorizer.transform(test.VF)

    other_features_subset_train = X_train[other_features]
    other_features_subset_test = X_test[other_features]
    other_features_subset_actual_test = test[other_features]

    X_train = sparse.hstack([
        vectorized_pos_train,
        vectorized_morph_train,
        other_features_subset_train
    ])

    X_test = sparse.hstack([
        vectorized_pos_test,
        vectorized_morph_test,
        other_features_subset_test
    ])


    X_actual_test = sparse.hstack([
        vectorized_pos_actual_test,
        vectorized_morph_actual_test,
        other_features_subset_actual_test
    ])


    model = MultiOutputRegressor(LGBMRegressor(n_jobs = -1))
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    
    test_prediction = model.predict(X_actual_test)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


Fold #0: 0.5368879184589069


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


Fold #1: 0.5403950112805512


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


Fold #2: 0.5414899838842435


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


Fold #3: 0.5377135881222913
Fold #4: 0.52903704439133
********************************************************************************

	Mean score: 0.5371


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


#### LGBM: tfidf + other features

In [91]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]

    text_vectorizer = TfidfVectorizer(ngram_range = (1,2))
    vectorized_text_train = text_vectorizer.fit_transform(X_train.full_text.tolist())
    vectorized_text_test = text_vectorizer.transform(X_test.full_text.tolist())
    vectorized_text_actual_test = text_vectorizer.transform(test.full_text.tolist())

    pos_vectorizer = TfidfVectorizer(ngram_range=(1,4))
    morph_vectorizer = TfidfVectorizer()

    vectorized_pos_train =  pos_vectorizer.fit_transform(X_train.poss_)
    vectorized_pos_test =  pos_vectorizer.transform(X_test.poss_)
    vectorized_pos_actual_test = pos_vectorizer.transform(test.poss_)

    vectorized_morph_train =  morph_vectorizer.fit_transform(X_train.VF)
    vectorized_morph_test =  morph_vectorizer.transform(X_test.VF)
    vectorized_morph_actual_test =  morph_vectorizer.transform(test.VF)

    other_features_subset_train = X_train[other_features]
    other_features_subset_test = X_test[other_features]
    other_features_subset_actual_test = test[other_features]

    X_train = sparse.hstack([
        vectorized_text_train,
        vectorized_pos_train,
        vectorized_morph_train,
        other_features_subset_train
    ])

    X_test = sparse.hstack([
        vectorized_text_test,
        vectorized_pos_test,
        vectorized_morph_test,
        other_features_subset_test
    ])


    X_actual_test = sparse.hstack([
        vectorized_text_actual_test,
        vectorized_pos_actual_test,
        vectorized_morph_actual_test,
        other_features_subset_actual_test
    ])


    model = MultiOutputRegressor(LGBMRegressor())
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    
    test_prediction = model.predict(X_actual_test)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


Fold #0: 0.5263995933051274


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


Fold #1: 0.5270824233516868


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


Fold #2: 0.526745088384568


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


Fold #3: 0.5260794824174228


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


Fold #4: 0.5176174900062112
********************************************************************************

	Mean score: 0.5248


In [29]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]

    text_vectorizer = TfidfVectorizer(ngram_range = (1,2))
    vectorized_text_train = text_vectorizer.fit_transform(X_train.full_text.tolist())
    vectorized_text_test = text_vectorizer.transform(X_test.full_text.tolist())
    vectorized_text_actual_test = text_vectorizer.transform(test.full_text.tolist())

    # pos_vectorizer = TfidfVectorizer(ngram_range=(1,4))
    # morph_vectorizer = TfidfVectorizer()

    # vectorized_pos_train =  pos_vectorizer.fit_transform(X_train.poss_)
    # vectorized_pos_test =  pos_vectorizer.transform(X_test.poss_)
    # vectorized_pos_actual_test = pos_vectorizer.transform(test.poss_)

    # vectorized_morph_train =  morph_vectorizer.fit_transform(X_train.VF)
    # vectorized_morph_test =  morph_vectorizer.transform(X_test.VF)
    # vectorized_morph_actual_test =  morph_vectorizer.transform(test.VF)

    other_features_subset_train = X_train[other_features]
    other_features_subset_test = X_test[other_features]
    other_features_subset_actual_test = test[other_features]

    X_train = sparse.hstack([
        vectorized_text_train,
        # vectorized_pos_train,
        # vectorized_morph_train,
        other_features_subset_train
    ])

    X_test = sparse.hstack([
        vectorized_text_test,
        # vectorized_pos_test,
        # vectorized_morph_test,
        other_features_subset_test
    ])


    X_actual_test = sparse.hstack([
        vectorized_text_actual_test,
        # vectorized_pos_actual_test,
        # vectorized_morph_actual_test,
        other_features_subset_actual_test
    ])


    model = MultiOutputRegressor(LGBMRegressor())
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    
    test_prediction = model.predict(X_actual_test)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


Fold #0: 0.5449932852683231


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


Fold #1: 0.536311281975003


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


Fold #2: 0.5381665761390868


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


Fold #3: 0.5318893976477513
Fold #4: 0.5305632985315182
********************************************************************************

	Mean score: 0.5364


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


In [30]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]

    text_vectorizer = TfidfVectorizer(ngram_range = (1,2))
    vectorized_text_train = text_vectorizer.fit_transform(X_train.full_text.tolist())
    vectorized_text_test = text_vectorizer.transform(X_test.full_text.tolist())
    vectorized_text_actual_test = text_vectorizer.transform(test.full_text.tolist())

    pos_vectorizer = TfidfVectorizer(ngram_range=(1,4))
    morph_vectorizer = TfidfVectorizer()

    vectorized_pos_train =  pos_vectorizer.fit_transform(X_train.poss_)
    vectorized_pos_test =  pos_vectorizer.transform(X_test.poss_)
    vectorized_pos_actual_test = pos_vectorizer.transform(test.poss_)

    vectorized_morph_train =  morph_vectorizer.fit_transform(X_train.VF)
    vectorized_morph_test =  morph_vectorizer.transform(X_test.VF)
    vectorized_morph_actual_test =  morph_vectorizer.transform(test.VF)

    # other_features_subset_train = X_train[other_features]
    # other_features_subset_test = X_test[other_features]
    # other_features_subset_actual_test = test[other_features]

    X_train = sparse.hstack([
        vectorized_text_train,
        vectorized_pos_train,
        vectorized_morph_train
        # other_features_subset_train
    ])

    X_test = sparse.hstack([
        vectorized_text_test,
        vectorized_pos_test,
        vectorized_morph_test,
        # other_features_subset_test
    ])


    X_actual_test = sparse.hstack([
        vectorized_text_actual_test,
        vectorized_pos_actual_test,
        vectorized_morph_actual_test
        # other_features_subset_actual_test
    ])


    model = MultiOutputRegressor(LGBMRegressor())
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    
    test_prediction = model.predict(X_actual_test)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


Fold #0: 0.5323105830126823


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


Fold #1: 0.5331513886951721


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


Fold #2: 0.5367470200323461


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


Fold #3: 0.5330283111925315


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


Fold #4: 0.5250341080636651
********************************************************************************

	Mean score: 0.5321


#### _XGB_

In [92]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]

    text_vectorizer = TfidfVectorizer(ngram_range = (1,2))
    vectorized_text_train = text_vectorizer.fit_transform(X_train.full_text.tolist())
    vectorized_text_test = text_vectorizer.transform(X_test.full_text.tolist())
    vectorized_text_actual_test = text_vectorizer.transform(test.full_text.tolist())

    pos_vectorizer = TfidfVectorizer(ngram_range=(1,4))
    morph_vectorizer = TfidfVectorizer()

    vectorized_pos_train =  pos_vectorizer.fit_transform(X_train.poss_)
    vectorized_pos_test =  pos_vectorizer.transform(X_test.poss_)
    vectorized_pos_actual_test = pos_vectorizer.transform(test.poss_)

    vectorized_morph_train =  morph_vectorizer.fit_transform(X_train.VF)
    vectorized_morph_test =  morph_vectorizer.transform(X_test.VF)
    vectorized_morph_actual_test =  morph_vectorizer.transform(test.VF)

    other_features_subset_train = X_train[other_features]
    other_features_subset_test = X_test[other_features]
    other_features_subset_actual_test = test[other_features]

    X_train = sparse.hstack([
        vectorized_text_train,
        vectorized_pos_train,
        vectorized_morph_train,
        other_features_subset_train
    ])

    X_test = sparse.hstack([
        vectorized_text_test,
        vectorized_pos_test,
        vectorized_morph_test,
        other_features_subset_test
    ])


    X_actual_test = sparse.hstack([
        vectorized_text_actual_test,
        vectorized_pos_actual_test,
        vectorized_morph_actual_test,
        other_features_subset_actual_test
    ])


    model = XGBRegressor(tree_method = 'gpu_hist')
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    
    test_prediction = model.predict(X_actual_test)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

Fold #0: 0.5611056418969932


KeyboardInterrupt: 

#### SVR

In [28]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]

    text_vectorizer = TfidfVectorizer(ngram_range = (1,2))
    vectorized_text_train = text_vectorizer.fit_transform(X_train.full_text.tolist())
    vectorized_text_test = text_vectorizer.transform(X_test.full_text.tolist())
    vectorized_text_actual_test = text_vectorizer.transform(test.full_text.tolist())

    pos_vectorizer = TfidfVectorizer(ngram_range=(1,4))
    morph_vectorizer = TfidfVectorizer()

    vectorized_pos_train =  pos_vectorizer.fit_transform(X_train.poss_)
    vectorized_pos_test =  pos_vectorizer.transform(X_test.poss_)
    vectorized_pos_actual_test = pos_vectorizer.transform(test.poss_)

    vectorized_morph_train =  morph_vectorizer.fit_transform(X_train.VF)
    vectorized_morph_test =  morph_vectorizer.transform(X_test.VF)
    vectorized_morph_actual_test =  morph_vectorizer.transform(test.VF)

    other_features_subset_train = X_train[other_features]
    other_features_subset_test = X_test[other_features]
    other_features_subset_actual_test = test[other_features]

    X_train = sparse.hstack([
        vectorized_text_train,
        vectorized_pos_train,
        vectorized_morph_train,
        other_features_subset_train
    ])

    X_test = sparse.hstack([
        vectorized_text_test,
        vectorized_pos_test,
        vectorized_morph_test,
        other_features_subset_test
    ])


    X_actual_test = sparse.hstack([
        vectorized_text_actual_test,
        vectorized_pos_actual_test,
        vectorized_morph_actual_test,
        other_features_subset_actual_test
    ])


    model = MultiOutputRegressor(SVR())
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    
    test_prediction = model.predict(X_actual_test)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

2022-09-15 07:55:22.899829: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Fold #0: 0.609460479500206
Fold #1: 0.6178598139568953
Fold #2: 0.5904801351429326
Fold #3: 0.5830384476681263
Fold #4: 0.5808801981592673
********************************************************************************

	Mean score: 0.5963


#### Submission

In [ ]:
preds_mean = np.mean(predictions, axis = 0)
submission = pd.DataFrame()
submission.loc[:,'text_id'] = test.text_id
submission.loc[:, TARGET_COLUMNS] = preds_mean

# submission.loc[:, TARGET_COLUMNS] = (submission.loc[:, TARGET_COLUMNS] * 2).round() / 2
submission

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.066927,2.759472,3.144001,3.051319,2.492089,2.823449
1,000BAD50D026,2.447473,2.554104,2.579586,2.329913,2.287815,2.528216
2,00367BB2546B,3.582943,3.397358,3.529512,3.458739,3.600916,3.366636


In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
# Score .54 LOL